# Together
Este notebook se encarga de juntar los algoritmos desarrollados hasta ahora

In [12]:
import pickle
from utils import CustomScaler, CustomMinMaxScaler

# Cargar el IPCA desde un archivo pickle
with open('ipca_low.pkl', 'rb') as f:
    loaded_ipca = pickle.load(f)

# Cargar el IPCA desde un archivo pickle
with open('minmaxscaler.pkl', 'rb') as f:
    minmaxscaler = pickle.load(f)
# Ahora puedes usar loaded_ipca como el objeto IncrementalPCA cargado


In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from grav_lens import get_datasets

import os

max_files = 1000
batch_size = 16
home_data = os.path.join("..","data")

train_dataset, val_dataset, test_dataset = get_datasets(
    data_index=1,
    max_files=max_files,
    home=home_data,
    batch_size=batch_size,
    val_split=0.2,
    test_split=0.1,
)

for X_batch, y_batch in train_dataset.take(1):
    print(X_batch.shape)
    print(y_batch.shape)

Using data folder: ..\data\1
(16, 128, 128, 3)
(16, 128, 128, 1)


In [7]:
y_standarized = (y_batch - mean) / std

In [8]:
mean_batch = tf.reduce_mean(y_standarized)
std_batch = tf.math.reduce_std(y_standarized)

print("Mean after standardization:", mean_batch.numpy())
print("Std after standardization:", std_batch.numpy())

Mean after standardization: -2595.1016
Std after standardization: 28356.662


In [10]:
mean_batch = tf.reduce_mean(y_batch)
std_batch = tf.math.reduce_std(y_batch)

print("Mean:", mean_batch.numpy())
print("Std:", std_batch.numpy())

Mean: -0.002720423
Std: 0.034414835


In [14]:
y_standard_scaler = standard_scaler.transform(y_batch)

mean_batch = tf.reduce_mean(y_standard_scaler)
std_batch = tf.math.reduce_std(y_standard_scaler)

print("Mean after standardization:", mean_batch.numpy())
print("Std after standardization:", std_batch.numpy())

Mean after standardization: -0.09035119
Std after standardization: 0.9488874


In [ ]:
class ExponentialWeightedStats:
    def __init__(self, alpha):
        self.alpha = alpha
        self.mu = 0
        self.S = 0
        self.initialized = False

    def update(self, x):
        if not self.initialized:
            self.mu = x
            self.S = 0
            self.initialized = True
        else:
            temp = x - self.mu
            new_mu = self.mu + self.alpha * temp
            self.S = self.alpha * self.S + (1 - self.alpha) * temp * (x - self.mu)
            self.mu = new_mu

    @property
    def mean(self):
        return self.mu

    @property
    def variance(self):
        return self.S

# Usage example
alpha = 0.05  # Weight factor
stats = ExponentialWeightedStats(alpha)
data_stream = [] # Data stream
for data_point in data_stream:
    stats.update(data_point)